# 01 Pre Augmentation Preparation

Before we can start with the augmentation, we need to prepare the dataset. 

In [1]:
import hashlib
import pandas as pd 

from utils import clean_text

from datasets import load_dataset, concatenate_datasets
from copy import deepcopy
from tqdm import tqdm

In [2]:
iapp    = load_dataset("iapp_wiki_qa_squad")
thaiqa  = load_dataset("thaiqa_squad")
xquad   = load_dataset("xquad", "xquad.th")
tydiqa = load_dataset("khalidalt/tydiqa-goldp", "thai")

Found cached dataset iapp_wiki_qa_squad (/Users/parinzee/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset thaiqa_squad (/Users/parinzee/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset xquad (/Users/parinzee/.cache/huggingface/datasets/xquad/xquad.th/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset tydiqa-goldp (/Users/parinzee/.cache/huggingface/datasets/khalidalt___tydiqa-goldp/thai/1.1.0/c0ba4613293c9a8e7cdc684809f0b2a520a66f38b23af4af5c09ea55e2b972f0)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Merge all splits of each dataset
def merge_dataset_splits(dataset):
    splits = list(dataset.keys())
    if len(splits) == 1:
        return dataset[splits[0]]
    else:
        return concatenate_datasets([dataset[split] for split in splits])

In [4]:
iapp = merge_dataset_splits(iapp).to_pandas()
thaiqa = merge_dataset_splits(thaiqa).to_pandas()
xquad = merge_dataset_splits(xquad).to_pandas()
tydiqa = merge_dataset_splits(tydiqa).to_pandas()

## Reformat the Datasets

In [5]:
iapp = iapp[['question', 'context', 'answers']]
thaiqa = thaiqa[['question', 'context', 'answers']]
xquad = xquad[['question', 'context', 'answers']]
tydiqa = tydiqa.rename(columns={'passage_text': 'context', "question_text": "question"})[['question', 'context', 'answers']]

In [6]:
# Datasets require more processing to make them all in the correct format
def get_offset_begin_position(cleaned_context, answers_text, answer_begin_positions):
    new_answer_begin_positions = []

    try:
        for answer_text, answer_begin_position in zip(answers_text, answer_begin_positions):
            # Find all all instances of the answer in cleaned_context
            possible_answer_begin_positions = [i for i in range(len(cleaned_context)) if cleaned_context.startswith(answer_text, i)] 

            if len(possible_answer_begin_positions) == 1:
                new_answer_begin_positions.append(possible_answer_begin_positions[0])
            elif len(possible_answer_begin_positions) > 1:
                # Find the closest answer to the original answer
                closest_answer_begin_position = min(possible_answer_begin_positions, key=lambda x:abs(x-answer_begin_position))
                new_answer_begin_positions.append(closest_answer_begin_position)
            else:
                raise Exception("No answer found in context")
    except:
        print("Error with answer: ", answer_text)
        print("Error with context: ", cleaned_context)
        print("Error with answer begin position: ", answer_begin_position)
        raise
    
    return new_answer_begin_positions

def process_row(row, answer_text_key, answer_start_key):
    # Normalize the text
    new_row = deepcopy(row)
    new_row["context"] = clean_text(row["context"])
    new_row["question"] = clean_text(row["question"], is_question=True)

    new_row["answers"] = {}
    new_row["answers"]["text"] = [clean_text(x) for x in row["answers"][answer_text_key]]

    # Reindex the dataset
    new_row["answers"]["answer_start"] = get_offset_begin_position(new_row["context"], new_row["answers"]["text"], row["answers"][answer_start_key])
    new_row["answers"]["answer_end"] = [x + len(y) for x, y in zip(new_row["answers"]["answer_start"], new_row["answers"]["text"])]

    return new_row

def sanity_check(datasets):
    # Match keys
    for dataset in tqdm(datasets):
        assert list(dataset.columns) == list(datasets[0].columns)
        assert dataset['answers'][0].keys() == datasets[0]['answers'][0].keys()
    
    print("All Keys Matched...")

    # Check theortical answers vs index
    for dataset in datasets:
        for _, row in tqdm(list(dataset.iterrows())):
            for text, begin, end in zip(row["answers"]['text'], row["answers"]['answer_start'], row["answers"]['answer_end']):
                assert text == row['context'][begin:end], f"Theoretical Answer: {text} | Indexed: {row['context'][begin:end]} | Context: {row['context']}"
    
    print("Theortical Answers Matched...")

    # Assert no empty columns
    for dataset in tqdm(datasets):
        assert not dataset.isnull().values.any()
    
    print("No empty columns...")

### Special Scenario for TyDiQA

Some of the following contexts (displayed below and discovered through cleaning & experimentation), have invalid answers—  thus we need to strip them out.
- 480
- 686
- 953
- 2057
- 3177

In [7]:
tydiqa[tydiqa["context"].str.startswith('โจแอนน์ "โจ"')]

,question,context,answers
480,เจ.เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
953,เจ. เค. โรว์ลิง มีผลงานเขียนเรื่องใดที่เป็นที่...,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': ['กกันดีในฐานะผู้ปร'], 'start_byte': ..."
2057,เจ. เค. โรว์ลิง คือใคร?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': ['ัลเบรธ (เกิด 31 กรกฎาคม'], 'start_b..."
3177,เจ. เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."


In [8]:
tydiqa[tydiqa["context"].str.startswith('โจแอนน์ "โจ"')].iloc[0]["context"]

'โจแอนน์ "โจ" โรว์ลิง (English: Joanne "Jo" Rowling, OBE FRSL[1]) หรือนามปากกา เจ. เค. โรว์ลิง[2] และ<b data-parsoid=\'{"dsr":[1509,1531,3,3]}\'>โรเบิร์ต กัลเบรธ (เกิด 31 กรกฎาคม ค.ศ. 1965)[3] เป็นนักเขียนนวนิยายชาวอังกฤษ ผู้เป็นที่รู้จักกันดีในฐานะผู้ประพันธ์วรรณกรรมแฟนตาซีชุด แฮร์รี่ พอตเตอร์ ซึ่งได้รับความความสนใจจากทั่วโลก ได้รับรางวัลมากมาย และมียอดขายกว่า 500 ล้านเล่ม[4] และยังเป็นหนังสือชุดที่ขายดีที่สุดในประวัติศาสตร์[5] ด้านภาพยนตร์ชุดแฮร์รี่ พอตเตอร์ที่ดัดแปลงมาจากหนังสือก็เป็นภาพยนตร์ชุดที่ทำรายได้มากที่สุดเป็นอันดับสองในประวัติศาสตร์[6] โรว์ลิงอนุมัติบทภาพยนตร์ทุกภาค[7] และตลอดจนควบคุมงานฝ่ายสร้างสรรค์ภาพยนตร์ภาคสุดท้ายในฐานะผู้อำนวยการสร้าง[8]'

In [9]:
tydiqa[tydiqa["context"].str.startswith('โจแอนน์ "โจ"')].iloc[1]["question"]

'เจ. เค. โรว์ลิง มีผลงานเขียนเรื่องใดที่เป็นที่รู้จักมากที่สุด ?'

In [10]:
clean_text(tydiqa[tydiqa["context"].str.startswith('โจแอนน์ "โจ"')].iloc[0]["context"])[150:] # This answer does not make sense at all

'นนวนิยายชาวอังกฤษ ผู้เป็นที่รู้จักกันดีในฐานะผู้ประพันธ์วรรณกรรมแฟนตาซีชุด แฮร์รี่ พอตเตอร์ ซึ่งได้รับความความสนใจจากทั่วโลก ได้รับรางวัลมากมาย และมียอดขายกว่า 500 ล้านเล่ม และยังเป็นหนังสือชุดที่ขายดีที่สุดในประวัติศาสตร์ ด้านภาพยนตร์ชุดแฮร์รี่ พอตเตอร์ที่ดัดแปลงมาจากหนังสือก็เป็นภาพยนตร์ชุดที่ทำรายได้มากที่สุดเป็นอันดับสองในประวัติศาสตร์ โรว์ลิงอนุมัติบทภาพยนตร์ทุกภาค และตลอดจนควบคุมงานฝ่ายสร้างสรรค์ภาพยนตร์ภาคสุดท้ายในฐานะผู้อำนวยการสร้าง'

In [11]:
tydiqa[tydiqa["context"].str.startswith("พระสิทธัตถพุทธเจ้า")] # Answer is incorrectly formatted

,question,context,answers
686,เจ้าชายสิทธัตถะ เกิดที่ไหน?,พระสิทธัตถพุทธเจ้า มีพระนามเดิมว่า<b data-pars...,"{'text': ['id='{""dsr"":[987'], 'start_byte': [2..."


#### Find all invalid answers

In [12]:
tydiqa[tydiqa["answers"].apply(lambda x: any(["}'>" in y for y in x["text"]]))]

,question,context,answers
480,เจ.เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
777,เรือหลวงจักรีนฤเบศร เริ่มสร้างเมื่อไหร่?,"เรือหลวง<i data-parsoid='{""dsr"":[5558,5573,2,2...","{'text': ['573,2,2]}'>จักรี'], 'start_byte': [..."
1766,กรมพระราชวังบวรสถานมงคล คืออะไร?,\nกรมพระราชวังบวรสถานมงคล แต่เรียกกันเป็นสามัญ...,"{'text': ['oid='{""dsr"":[764,777,3,3]}'>วังหน้า..."
2063,เซอร์ วิลเลียม วอลเลซ เกิดที่ไหน ?,"สถานที่เกิดและวันเกิดของ<b data-parsoid='{""dsr...","{'text': ['}'>เซอร์'], 'start_byte': [111], 'l..."
2124,อัศวินเทมพลาร์ คืออะไร?,ทหารผู้ยากแห่งพระคริสต์และพระวิหารแห่งโซโลมอน ...,"{'text': ['dsr"":[1812,1833,3,3]}'>คณะแห่งพระวิ..."
3177,เจ. เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
4015,เสรีภาพ คืออะไร ?,"เสรีภาพ, อิสรภาพ หรือ<b data-parsoid='{""dsr"":[...","{'text': ['60,3,3]}'>เสร', '0,3,3]}'>เสร', '0,..."
4132,งานวิจัย คืออะไร ?,การวิจัย (English: research) หมายถึงการกระทำขอ...,{'text': ['การกระทำของมนุษย์เพื่อค้นหาความจริง...
4201,ราษฎรเรียกสมเด็จพระเจ้าอุทุมพร ว่าอะไร?,สมเด็จพระเจ้าอุทุมพร มีพระนามเดิมว่า<b data-pa...,"{'text': ['7,2597,3,3]}'>เจ้', '7,2597,3,3]}'>..."
4360,คุณหญิงกนิษฐา วิเชียรเจริญ เกิดเมื่อไหร่?,"\n\n\nคุณหญิง<b data-parsoid='{""dsr"":[50,75,3,...","{'text': ['r"":[50,75,3,3]}'>กนิษฐ', 'r"":[50,75..."


In [13]:
tydiqa[tydiqa["answers"].apply(lambda x: any([":[" in y for y in x["text"]]))]

,question,context,answers
25,ค.ศ. 1951 ใครเป็นนายกรัฐมนตรีของอังกฤษ ?,"เซอร์<b data-parsoid='{""dsr"":[2047,2089,3,3]}'...","{'text': ['เซอร์<b data-parsoid='{""dsr"":[2047,..."
480,เจ.เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
686,เจ้าชายสิทธัตถะ เกิดที่ไหน?,พระสิทธัตถพุทธเจ้า มีพระนามเดิมว่า<b data-pars...,"{'text': ['id='{""dsr"":[987'], 'start_byte': [2..."
883,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,{'text': ['ออกัสเตนบอร์ก แถบชานเมือง<i data-p...
1354,ศาสดาของศาสนาฮินดูคือใคร?,\nศาสนาฮินดู (English: Hinduism) หรือ สนาตนธรร...,"{'text': ['dsr"":[257,281'], 'start_byte': [332..."
1766,กรมพระราชวังบวรสถานมงคล คืออะไร?,\nกรมพระราชวังบวรสถานมงคล แต่เรียกกันเป็นสามัญ...,"{'text': ['oid='{""dsr"":[764,777,3,3]}'>วังหน้า..."
2124,อัศวินเทมพลาร์ คืออะไร?,ทหารผู้ยากแห่งพระคริสต์และพระวิหารแห่งโซโลมอน ...,"{'text': ['dsr"":[1812,1833,3,3]}'>คณะแห่งพระวิ..."
2564,ศาสดาของ ศาสนาพราหมณ์-ฮินดู คือใคร?,\nศาสนาฮินดู (English: Hinduism) หรือ สนาตนธรร...,"{'text': ['dsr"":[257,281'], 'start_byte': [332..."
3177,เจ. เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
3957,วอยซ์ ทีวี ดำเนินการโดยบริษัทใด?,บริษัท วอยซ์ ทีวี จำกัด จดทะเบียนจัดตั้งขึ้น เ...,"{'text': ['data-parsoid='{""dsr"":[4218,42', 'ั้..."


In [14]:
tydiqa[tydiqa["answers"].apply(lambda x: any(["=" in y for y in x["text"]]))]

,question,context,answers
25,ค.ศ. 1951 ใครเป็นนายกรัฐมนตรีของอังกฤษ ?,"เซอร์<b data-parsoid='{""dsr"":[2047,2089,3,3]}'...","{'text': ['เซอร์<b data-parsoid='{""dsr"":[2047,..."
686,เจ้าชายสิทธัตถะ เกิดที่ไหน?,พระสิทธัตถพุทธเจ้า มีพระนามเดิมว่า<b data-pars...,"{'text': ['id='{""dsr"":[987'], 'start_byte': [2..."
883,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,{'text': ['ออกัสเตนบอร์ก แถบชานเมือง<i data-p...
1524,แนวร่วมประชาธิปไตยมีเสื้อสีอะไรเป็นสัญลักษณ์?,แนวร่วมประชาธิปไตยต่อต้านเผด็จการแห่งชาติ (ชื่...,"{'text': ['oid='], 'start_byte': [824], 'limit..."
1766,กรมพระราชวังบวรสถานมงคล คืออะไร?,\nกรมพระราชวังบวรสถานมงคล แต่เรียกกันเป็นสามัญ...,"{'text': ['oid='{""dsr"":[764,777,3,3]}'>วังหน้า..."
1959,ปฏิกิริยานิวเคลียร์ฟิวชั่น เกิดขึ้นได้อย่างไร?,\nพลังงานฟิวชั่น (English: Fusion power) คือพล...,{'text': ['นิวเคลียสของอะตอมธาตุเบาหลอมตัวเข้า...
3528,ใครเป็นผู้คิดค้น สเตรปโตมัยซิน?,สเตรปโตมัยซินถูกค้นพบครั้งแรกเมื่อวันที่ 19 ตุ...,"{'text': ['<b data-parsoid='], 'start_byte': [..."
3957,วอยซ์ ทีวี ดำเนินการโดยบริษัทใด?,บริษัท วอยซ์ ทีวี จำกัด จดทะเบียนจัดตั้งขึ้น เ...,"{'text': ['data-parsoid='{""dsr"":[4218,42', 'ั้..."
4132,งานวิจัย คืออะไร ?,การวิจัย (English: research) หมายถึงการกระทำขอ...,{'text': ['การกระทำของมนุษย์เพื่อค้นหาความจริง...


In [15]:
tydiqa[tydiqa["answers"].apply(lambda x: any(["[" in y for y in x["text"]]))]

,question,context,answers
25,ค.ศ. 1951 ใครเป็นนายกรัฐมนตรีของอังกฤษ ?,"เซอร์<b data-parsoid='{""dsr"":[2047,2089,3,3]}'...","{'text': ['เซอร์<b data-parsoid='{""dsr"":[2047,..."
395,ลัทธิขงจื๊อคืออะไร?,thumbnail|250px|หอต้าเฉิง (Dàchéng) อาคารหลักข...,"{'text': ['นาขงจื๊อ[1] (English: Confucian'], ..."
480,เจ.เค. โรว์ลิ่ง เกิดวันที่เท่าไหร่?,"โจแอนน์ ""โจ"" โรว์ลิง (English: Joanne ""Jo"" Row...","{'text': [':[1509,1531,3,3]}'>โ'], 'start_byte..."
543,พระไตรปิฎก เป็นคำสอนของใคร?,\nพระไตรปิฎก (Pali: Tipiṭaka; Sanskrit: त्रिपि...,"{'text': ['พระโคตมพุทธเจ้า['], 'start_byte': [..."
683,ประพุทธเจ้าทรงประสูติเมื่อวันที่เท่าไหร่?,\nในคืนที่พระโพธิสัตว์เสด็จปฏิสนธิในครรภ์พระนา...,{'text': ['[5]ขึ้น 15 ค่ำ เดือน 6 ปีจอ ก่อนพุท...
686,เจ้าชายสิทธัตถะ เกิดที่ไหน?,พระสิทธัตถพุทธเจ้า มีพระนามเดิมว่า<b data-pars...,"{'text': ['id='{""dsr"":[987'], 'start_byte': [2..."
883,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,หลังคาเขียวที่เชื่อว่าเป็นสวนพฤกษศาสตร์หลังคาเ...,{'text': ['ออกัสเตนบอร์ก แถบชานเมือง<i data-p...
1354,ศาสดาของศาสนาฮินดูคือใคร?,\nศาสนาฮินดู (English: Hinduism) หรือ สนาตนธรร...,"{'text': ['dsr"":[257,281'], 'start_byte': [332..."
1617,นาย ฟิเนียส์ พี. เกจ เกิดที่ไหน ?,\nนายเกจเป็นบุตรคนแรกในพี่น้อง 5 คนเกิดจากนายเ...,{'text': ['งเทศมณฑลกราฟตัน มลรัฐนิวแฮมป์เชียร...
1766,กรมพระราชวังบวรสถานมงคล คืออะไร?,\nกรมพระราชวังบวรสถานมงคล แต่เรียกกันเป็นสามัญ...,"{'text': ['oid='{""dsr"":[764,777,3,3]}'>วังหน้า..."


In [16]:
tydiqa = tydiqa.drop([480, 686, 953, 2057, 3177])
tydiqa = tydiqa[tydiqa["answers"].apply(lambda x: not any(["}'>" in y for y in x["text"]]))]
tydiqa = tydiqa[tydiqa["answers"].apply(lambda x: not any([":[" in y for y in x["text"]]))]
tydiqa = tydiqa[tydiqa["answers"].apply(lambda x: not any(["=" in y for y in x["text"]]))]
tydiqa = tydiqa[tydiqa["answers"].apply(lambda x: not any(["[" in y for y in x["text"]]))]

### Apply cleaning process to dataset

In [17]:
iapp = iapp.apply(lambda x: process_row(x, "text", "answer_start"), axis=1)
thaiqa = thaiqa.apply(lambda x: process_row(x, "answer", "answer_begin_position"), axis=1)
xquad = xquad.apply(lambda x: process_row(x, "text", "answer_start"), axis=1)
tydiqa = tydiqa.apply(lambda x: process_row(x, "text", "start_byte"), axis=1)

/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
/Users/parinzee/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/utils.py:7: MarkupRese

In [18]:
sanity_check([iapp, thaiqa, xquad, tydiqa])

100%|██████████| 4/4 [00:00<00:00, 5155.87it/s]


All Keys Matched...


100%|██████████| 4542/4542 [00:00<00:00, 163147.14it/s]


Theortical Answers Matched...


100%|██████████| 4/4 [00:00<00:00, 1241.47it/s]

No empty columns...


## Merge the Datasets

In [19]:
iapp["source"] = "iapp"
thaiqa["source"] = "thaiqa"
xquad["source"] = "xquad"
tydiqa["source"] = "tydiqa"

In [20]:
final = pd.concat([iapp, thaiqa, xquad, tydiqa], ignore_index=True)
final

,question,context,answers,source
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp
4,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp
...,...,...,...,...
17043,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa
17044,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa
17045,ยู ซึง-โฮ เริ่มเข้าวงการบันเทิงเมื่อไหร่?,ยู ซึง-โฮเดบิวต์เมื่อปี 1999 กับผลงานโฆษณาของ ...,"{'text': ['ปี 1999', '1999'], 'answer_start': ...",tydiqa
17046,นภัทร อินทร์ใจเอื้อมีมารดาชื่อว่าอะไร?,กันเกิดเมื่อวันที่ 23 ตุลาคม พ.ศ. 2533 จังหวัด...,"{'text': ['นางวรรณา อินทร์ใจเอื้อ', 'นางวรรณา ...",tydiqa


In [21]:
# Drop duplicates to avoid accidentally leaking data into the test set
print(f"Before dropping duplicates: {len(final)}")
print(f"Values per source: {final['source'].value_counts()}")
print("")

final = final.drop_duplicates(subset=["context", "question"], keep="first").reset_index(drop=True)

print(f"After dropping duplicates: {len(final)}")
print(f"Values per source: {final['source'].value_counts()}")

Before dropping duplicates: 17048
Values per source: source
iapp      7242
tydiqa    4542
thaiqa    4074
xquad     1190
Name: count, dtype: int64

After dropping duplicates: 16980
Values per source: source
iapp      7226
tydiqa    4523
thaiqa    4051
xquad     1180
Name: count, dtype: int64


In [22]:
# Generate unique ids for each row by hashing the context and question
final["id"] = final.apply(lambda x: hashlib.sha256((x["context"] + x["question"]).encode("utf-8")).hexdigest(), axis=1)

In [23]:
final.to_csv("data/01_prepare_dataset.csv", index=False)

Next steps:
- Translate and back translate the dataset with Google Sheets to and from English (save to `data/02_prepare_dataset_translated.csv`)
- Perform Thai Augmentation (`02_Thai_Augmentation.ipynb`)